In [1]:
# Importing necessary modules
import gc
import copy
import os
import time
import numpy as np
import pandas as pd
from sklearn.model_selection import KFold
from sklearn.metrics import r2_score, mean_squared_error
from model_data_definitions import *
from concurrent.futures import ThreadPoolExecutor, ProcessPoolExecutor
import os
import numpy as np

In [2]:
def modelNames():

    models = ["Linear Model",
              "Wide ReLU ANN",
              "Deep ReLU ANN", 
              "One Parameter"
    ]

    for partition_num in [1,2,4,8,10]:
        models.append(f"Spline ANN (z={partition_num})")
        models.append(f"Lookup Table (z={partition_num})")
        models.append(f"ABEL-Spline (z={partition_num})")

    return models

def modelNames():

    models = ["Wide ReLU ANN",
              "Deep ReLU ANN", 
              "Linear Model"             
             ]
    for partition_num in [1,2,4,8,10]: 
        models.append(f"ABEL-Spline (z={partition_num})")
        
    for partition_num in [1,2,4,8,10]: 
        models.append(f"Spline ANN (z={partition_num})")
    
        
    for partition_num in [1,2,4,8,10]: 
        models.append(f"Lookup Table (z={partition_num})")

    return models

model_names = modelNames()

print(model_names)

['Wide ReLU ANN', 'Deep ReLU ANN', 'Linear Model', 'ABEL-Spline (z=1)', 'ABEL-Spline (z=2)', 'ABEL-Spline (z=4)', 'ABEL-Spline (z=8)', 'ABEL-Spline (z=10)', 'Spline ANN (z=1)', 'Spline ANN (z=2)', 'Spline ANN (z=4)', 'Spline ANN (z=8)', 'Spline ANN (z=10)', 'Lookup Table (z=1)', 'Lookup Table (z=2)', 'Lookup Table (z=4)', 'Lookup Table (z=8)', 'Lookup Table (z=10)']


In [3]:
#np.save('data_dictionary_100_epochs_10_folds.npy', data)
data = np.load('data_dictionary_100_epochs_10_folds.npy', allow_pickle=True).item()

In [4]:
# Function to calculate mean and standard deviation of a metric for each fold for each model, over all datasets
def calculate_mean_std_metric(metric):
    # Initialize dictionary to hold means and standard deviations
    means_stds = {}

    # Iterate over datasets
    for dataset_name in data:
        # Iterate over models
        for model_name in data[dataset_name]:
            # Initialize list to hold metric values for this model
            metric_values = []

            # Iterate over folds
            for fold_number in data[dataset_name][model_name]:
                # Add metric value to list
                metric_values.append(data[dataset_name][model_name][fold_number][metric])

            # Calculate mean and standard deviation and add to dictionary
            mean_metric = np.mean(metric_values)
            std_metric = np.std(metric_values)
            
            if dataset_name not in means_stds:
                means_stds[dataset_name] = {}
                
            if model_name not in means_stds[dataset_name]:
                means_stds[dataset_name][model_name] = {}

            means_stds[dataset_name][model_name]['mean'] = mean_metric
            means_stds[dataset_name][model_name]['std'] = std_metric

    return means_stds

# Calculate mean and standard deviation of r_squared value for each fold for each model, over all datasets
#mean_std_r_squared_values = calculate_mean_std_metric('r_squared_value')
#print(mean_std_r_squared_values)

# Calculate mean and standard deviation of test_error value for each fold for each model, over all datasets
#mean_std_test_error_values = calculate_mean_std_metric('test_error')
#print(mean_std_test_error_values)

# Calculate mean and standard deviation of loss value for each fold for each model, over all datasets
#mean_std_loss_values = calculate_mean_std_metric('loss')
#print(mean_std_loss_values)

In [5]:
filtered_datasets_metadata, datasets = fetch_return_filtered_pmlb_data_sets()
filtered_datasets_metadata

,dataset,n_instances,n_features,n_binary_features,n_categorical_features,n_continuous_features,endpoint_type,n_classes,imbalance,task
0,1027_ESL,488,4,0,0,4,continuous,9.0,0.099363,regression
2,1029_LEV,1000,4,0,0,4,continuous,5.0,0.111245,regression
3,1030_ERA,1000,4,0,0,4,continuous,9.0,0.031251,regression
5,1096_FacultySalaries,50,4,0,0,4,continuous,39.0,0.004063,regression
13,192_vineyard,52,2,0,0,2,continuous,19.0,0.040475,regression
23,228_elusage,55,2,0,0,2,continuous,52.0,0.000953,regression
25,230_machine_cpu,209,6,0,0,6,continuous,116.0,0.004906,regression
29,485_analcatdata_vehicle,48,4,0,0,4,continuous,47.0,0.000434,regression
32,519_vinnie,380,2,0,0,2,continuous,16.0,0.030146,regression
34,523_analcatdata_neavote,100,2,0,0,2,continuous,8.0,0.136914,regression


In [12]:
import matplotlib.pyplot as plt
import numpy as np
from mpl_toolkits.axes_grid1 import make_axes_locatable

def modelNames():

    models = ["Linear Model",
              "Wide ReLU ANN",
              "Deep ReLU ANN"
             ]
        
    for partition_num in [1,2,4,8,10]: 
        models.append(f"ABEL-Spline (z={partition_num})")
        
    for partition_num in [1,2,4,8,10]: 
        models.append(f"Spline ANN (z={partition_num})")
        
    for partition_num in [1,2,4,8,10]: 
        models.append(f"Lookup Table (z={partition_num})")

    return models

model_names = modelNames()

print(model_names)

def plot_metric(metric):
    metrics_labels = {
    'loss': 'MAE',
    'r_squared_value': '$R^2$',
    'test_error': 'MSE'}
    instances_dict = filtered_datasets_metadata.set_index('dataset')['n_instances'].to_dict()
    features_dict = filtered_datasets_metadata.set_index('dataset')['n_features'].to_dict()

    unique_features_counts = sorted(list(set(features_dict.values())))

    sorted_dataset_names = []
    bracket_positions = {}

    for feature_count in unique_features_counts:
        datasets_with_feature_count = [dataset for dataset, features in features_dict.items() if features == feature_count]
        sorted_datasets = sorted(datasets_with_feature_count, key=lambda x: instances_dict[x])

        if not sorted_datasets:
            continue

        start_index = len(sorted_dataset_names)
        end_index = start_index + len(sorted_datasets) - 1

        bracket_positions[feature_count] = (start_index, end_index)
        
        sorted_dataset_names += [(dataset, instances_dict[dataset]) for dataset in sorted_datasets]

    data_array = np.zeros((len(model_names), len(sorted_dataset_names)))    

    mean_std_of_metric = calculate_mean_std_metric(metric)

    for i, model_name in enumerate(model_names):
        for j, (dataset_name, _) in enumerate(sorted_dataset_names):
            if model_name in mean_std_of_metric[dataset_name]:
                data_array[i, j] = mean_std_of_metric[dataset_name][model_name]['mean']
                '''
                if metric == 'r_squared_value':
                    data_array[i, j] = mean_std_of_metric[dataset_name][model_name]['mean']
                else:
                    data_array[i, j] = np.log(mean_std_of_metric[dataset_name][model_name]['mean'])
                    '''
    fig, ax = plt.subplots(figsize=(10, len(model_names)/2))
    divider = make_axes_locatable(ax)
    cax = divider.append_axes("right", size="5%", pad=0.05)

    cmap_choice = 'RdYlBu' if metric == 'r_squared_value' else 'viridis'

    # Set color limits for R^2
    clim=(-1, 1) if metric=='r_squared_value' else None

    if clim:
        im = ax.imshow(data_array, interpolation='nearest', cmap=cmap_choice, vmin=clim[0], vmax=clim[1])
    else:
        im = ax.imshow(data_array, interpolation='nearest', cmap=cmap_choice)
        
    cbar = fig.colorbar(im, cax=cax, orientation="vertical")

    cbar.set_label(metrics_labels[metric], rotation=90) 

    ax.set_yticks(np.arange(len(model_names)))
    ax.set_yticklabels(model_names)

    ax.set_xticks(np.arange(len(sorted_dataset_names)))
    ax.set_xticklabels([f"{name} ({n_instances})" for name, n_instances in sorted_dataset_names], rotation=90)

    for feature_count, (start_index, end_index) in bracket_positions.items():
        ax.annotate("", xy=(start_index-0.5,-0.5), xycoords='data',
                      xytext=(end_index+0.5,-0.5), textcoords='data',
                      annotation_clip=False,
                      arrowprops=dict(arrowstyle="-",
                                      shrinkA=0, shrinkB=0,
                                      connectionstyle="angle3"))
        ax.text((start_index+end_index)/2, -1.5, f"{feature_count}D",
                  horizontalalignment='center', verticalalignment='top')

        if start_index > 0:
            ax.axvline(x=start_index-0.5, color='white', linewidth=1)

    #plt.savefig(f'{metric}.png', dpi=300, bbox_inches='tight')
    #plt.close(fig)  # close the figure
    plt.savefig(f'{metric}.png', dpi=300, bbox_inches='tight')
    plt.close(fig)  # close the figure
    # Show plot
    #plt.show()

# Now you can call this function to plot different metrics
plot_metric('r_squared_value')
plot_metric('loss')
plot_metric('test_error')

['Linear Model', 'Wide ReLU ANN', 'Deep ReLU ANN', 'ABEL-Spline (z=1)', 'ABEL-Spline (z=2)', 'ABEL-Spline (z=4)', 'ABEL-Spline (z=8)', 'ABEL-Spline (z=10)', 'Spline ANN (z=1)', 'Spline ANN (z=2)', 'Spline ANN (z=4)', 'Spline ANN (z=8)', 'Spline ANN (z=10)', 'Lookup Table (z=1)', 'Lookup Table (z=2)', 'Lookup Table (z=4)', 'Lookup Table (z=8)', 'Lookup Table (z=10)']


# Log values

In [13]:
import matplotlib.pyplot as plt
import numpy as np
from mpl_toolkits.axes_grid1 import make_axes_locatable

def modelNames():

    models = ["Linear Model",
              "Wide ReLU ANN",
              "Deep ReLU ANN"
             ]
        
    for partition_num in [1,2,4,8,10]: 
        models.append(f"ABEL-Spline (z={partition_num})")
        
    for partition_num in [1,2,4,8,10]: 
        models.append(f"Spline ANN (z={partition_num})")
        
    for partition_num in [1,2,4,8,10]: 
        models.append(f"Lookup Table (z={partition_num})")

    return models

model_names = modelNames()

print(model_names)

def plot_metric(metric):
    
    # Metrics labels dictionary
    metrics_labels = {
    'loss': 'log MAE',
    'r_squared_value': '$R^2$',
    'test_error': 'log MSE'
    }
    
    instances_dict = filtered_datasets_metadata.set_index('dataset')['n_instances'].to_dict()
    features_dict = filtered_datasets_metadata.set_index('dataset')['n_features'].to_dict()

    unique_features_counts = sorted(list(set(features_dict.values())))

    sorted_dataset_names = []
    bracket_positions = {}

    for feature_count in unique_features_counts:
        datasets_with_feature_count = [dataset for dataset, features in features_dict.items() if features == feature_count]
        sorted_datasets = sorted(datasets_with_feature_count, key=lambda x: instances_dict[x])

        if not sorted_datasets:
            continue

        start_index = len(sorted_dataset_names)
        end_index = start_index + len(sorted_datasets) - 1

        bracket_positions[feature_count] = (start_index, end_index)
        
        sorted_dataset_names += [(dataset, instances_dict[dataset]) for dataset in sorted_datasets]

    data_array = np.zeros((len(model_names), len(sorted_dataset_names)))    

    mean_std_of_metric = calculate_mean_std_metric(metric)

    for i, model_name in enumerate(model_names):
        for j, (dataset_name, _) in enumerate(sorted_dataset_names):
            if model_name in mean_std_of_metric[dataset_name]:
                if metric == 'r_squared_value':
                    data_array[i, j] = mean_std_of_metric[dataset_name][model_name]['mean']
                else:
                    data_array[i, j] = np.log(mean_std_of_metric[dataset_name][model_name]['mean'])

    fig, ax = plt.subplots(figsize=(10, len(model_names)/2))
    divider = make_axes_locatable(ax)
    cax = divider.append_axes("right", size="5%", pad=0.05)

    cmap_choice = 'RdYlBu' if metric == 'r_squared_value' else 'viridis'

    # Set color limits for R^2
    clim=(-1, 1) if metric=='r_squared_value' else None

    if clim:
        im = ax.imshow(data_array, interpolation='nearest', cmap=cmap_choice, vmin=clim[0], vmax=clim[1])
    else:
        im = ax.imshow(data_array, interpolation='nearest', cmap=cmap_choice)
        
    cbar = fig.colorbar(im, cax=cax, orientation="vertical")

    cbar.set_label(metrics_labels[metric], rotation=90) 

    ax.set_yticks(np.arange(len(model_names)))
    ax.set_yticklabels(model_names)

    ax.set_xticks(np.arange(len(sorted_dataset_names)))
    ax.set_xticklabels([f"{name} ({n_instances})" for name, n_instances in sorted_dataset_names], rotation=90)

    for feature_count, (start_index, end_index) in bracket_positions.items():
        ax.annotate("", xy=(start_index-0.5,-0.5), xycoords='data',
                      xytext=(end_index+0.5,-0.5), textcoords='data',
                      annotation_clip=False,
                      arrowprops=dict(arrowstyle="-",
                                      shrinkA=0, shrinkB=0,
                                      connectionstyle="angle3"))
        ax.text((start_index+end_index)/2, -1.5, f"{feature_count}D",
                  horizontalalignment='center', verticalalignment='top')

        if start_index > 0:
            ax.axvline(x=start_index-0.5, color='white', linewidth=1)

    #plt.savefig(f'{metric}.png', dpi=300, bbox_inches='tight')
    #plt.close(fig)  # close the figure
    if metric != 'r_squared_value':
        plt.savefig(f'{metric}_log.png', dpi=300, bbox_inches='tight')
    else:
        plt.savefig(f'{metric}.png', dpi=300, bbox_inches='tight')
    plt.close(fig)  # close the figure
    # Show plot
    plt.show()

# Now you can call this function to plot different metrics
plot_metric('r_squared_value')
plot_metric('loss')
plot_metric('test_error')

['Linear Model', 'Wide ReLU ANN', 'Deep ReLU ANN', 'ABEL-Spline (z=1)', 'ABEL-Spline (z=2)', 'ABEL-Spline (z=4)', 'ABEL-Spline (z=8)', 'ABEL-Spline (z=10)', 'Spline ANN (z=1)', 'Spline ANN (z=2)', 'Spline ANN (z=4)', 'Spline ANN (z=8)', 'Spline ANN (z=10)', 'Lookup Table (z=1)', 'Lookup Table (z=2)', 'Lookup Table (z=4)', 'Lookup Table (z=8)', 'Lookup Table (z=10)']


# Plot training MAE

In [14]:
data['1027_ESL']['ABEL-Spline (z=10)']['1']['train_history'][-1]

0.25964778661727905

In [15]:
# Function to calculate mean and standard deviation of a metric for each fold for each model, over all datasets
def calculate_mean_std_metric(metric):
    # Initialize dictionary to hold means and standard deviations
    means_stds = {}

    # Iterate over datasets
    for dataset_name in data:
        # Iterate over models
        for model_name in data[dataset_name]:
            # Initialize list to hold metric values for this model
            metric_values = []

            # Iterate over folds
            for fold_number in data[dataset_name][model_name]:
                # Add metric value to list
                if metric == 'train_history':
                    metric_values.append(data[dataset_name][model_name][fold_number][metric][-1])
                else:
                    metric_values.append(data[dataset_name][model_name][fold_number][metric])

            # Calculate mean and standard deviation and add to dictionary
            mean_metric = np.mean(metric_values)
            std_metric = np.std(metric_values)
            
            if dataset_name not in means_stds:
                means_stds[dataset_name] = {}
                
            if model_name not in means_stds[dataset_name]:
                means_stds[dataset_name][model_name] = {}

            means_stds[dataset_name][model_name]['mean'] = mean_metric
            means_stds[dataset_name][model_name]['std'] = std_metric

    return means_stds

# Calculate mean and standard deviation of r_squared value for each fold for each model, over all datasets
#mean_std_r_squared_values = calculate_mean_std_metric('r_squared_value')
#print(mean_std_r_squared_values)

# Calculate mean and standard deviation of test_error value for each fold for each model, over all datasets
#mean_std_test_error_values = calculate_mean_std_metric('test_error')
#print(mean_std_test_error_values)

# Calculate mean and standard deviation of loss value for each fold for each model, over all datasets
#mean_std_loss_values = calculate_mean_std_metric('loss')
#print(mean_std_loss_values)

In [16]:
calculate_mean_std_metric('train_history')

{'1027_ESL': {'ABEL-Spline (z=1)': {'mean': 0.26704726219177244,
   'std': 0.0034103409496238544},
  'ABEL-Spline (z=10)': {'mean': 0.2557650491595268,
   'std': 0.0033383137850338303},
  'ABEL-Spline (z=2)': {'mean': 0.2602288544178009,
   'std': 0.0034655342844600806},
  'ABEL-Spline (z=4)': {'mean': 0.25806793570518494,
   'std': 0.003119272076416409},
  'ABEL-Spline (z=8)': {'mean': 0.2564000517129898,
   'std': 0.003691158690167444},
  'Deep ReLU ANN': {'mean': 0.20907655954360962, 'std': 0.008047117169300379},
  'Linear Model': {'mean': 0.30961486101150515, 'std': 0.015474452264007212},
  'Lookup Table (z=1)': {'mean': 0.7967861533164978,
   'std': 0.003742144154538692},
  'Lookup Table (z=10)': {'mean': 0.42519529163837433,
   'std': 0.002903927181251129},
  'Lookup Table (z=2)': {'mean': 0.4084880769252777,
   'std': 0.0037162737911104545},
  'Lookup Table (z=4)': {'mean': 0.36656316220760343,
   'std': 0.003803010378784758},
  'Lookup Table (z=8)': {'mean': 0.4126876085996628,

In [24]:
import matplotlib.pyplot as plt
import numpy as np
from mpl_toolkits.axes_grid1 import make_axes_locatable

def plot_metric(metric):
    metrics_labels = {
    'loss': 'MAE',
    'r_squared_value': '$R^2$',
    'test_error': 'MSE',
    'train_history': 'Training MAE'}
    instances_dict = filtered_datasets_metadata.set_index('dataset')['n_instances'].to_dict()
    features_dict = filtered_datasets_metadata.set_index('dataset')['n_features'].to_dict()

    unique_features_counts = sorted(list(set(features_dict.values())))

    sorted_dataset_names = []
    bracket_positions = {}

    for feature_count in unique_features_counts:
        datasets_with_feature_count = [dataset for dataset, features in features_dict.items() if features == feature_count]
        sorted_datasets = sorted(datasets_with_feature_count, key=lambda x: instances_dict[x])

        if not sorted_datasets:
            continue

        start_index = len(sorted_dataset_names)
        end_index = start_index + len(sorted_datasets) - 1

        bracket_positions[feature_count] = (start_index, end_index)
        
        sorted_dataset_names += [(dataset, instances_dict[dataset]) for dataset in sorted_datasets]

    data_array = np.zeros((len(model_names), len(sorted_dataset_names)))    

    mean_std_of_metric = calculate_mean_std_metric(metric)

    for i, model_name in enumerate(model_names):
        for j, (dataset_name, _) in enumerate(sorted_dataset_names):
            if model_name in mean_std_of_metric[dataset_name]:
                data_array[i, j] = mean_std_of_metric[dataset_name][model_name]['mean']
                '''
                if metric == 'r_squared_value':
                    data_array[i, j] = mean_std_of_metric[dataset_name][model_name]['mean']
                else:
                    data_array[i, j] = np.log(mean_std_of_metric[dataset_name][model_name]['mean'])
                    '''
    fig, ax = plt.subplots(figsize=(10, len(model_names)/2))
    divider = make_axes_locatable(ax)
    cax = divider.append_axes("right", size="5%", pad=0.05)

    cmap_choice = 'RdYlBu' if metric == 'r_squared_value' else 'viridis'

    # Set color limits for R^2
    clim=(-1, 1) if metric=='r_squared_value' else None

    if clim:
        im = ax.imshow(data_array, interpolation='nearest', cmap=cmap_choice, vmin=clim[0], vmax=clim[1])
    else:
        im = ax.imshow(data_array, interpolation='nearest', cmap=cmap_choice)
        
    cbar = fig.colorbar(im, cax=cax, orientation="vertical")

    cbar.set_label(metrics_labels[metric], rotation=90) 

    ax.set_yticks(np.arange(len(model_names)))
    ax.set_yticklabels(model_names)

    ax.set_xticks(np.arange(len(sorted_dataset_names)))
    ax.set_xticklabels([f"{name} ({n_instances})" for name, n_instances in sorted_dataset_names], rotation=90)

    for feature_count, (start_index, end_index) in bracket_positions.items():
        ax.annotate("", xy=(start_index-0.5,-0.5), xycoords='data',
                      xytext=(end_index+0.5,-0.5), textcoords='data',
                      annotation_clip=False,
                      arrowprops=dict(arrowstyle="-",
                                      shrinkA=0, shrinkB=0,
                                      connectionstyle="angle3"))
        ax.text((start_index+end_index)/2, -1.5, f"{feature_count}D",
                  horizontalalignment='center', verticalalignment='top')

        if start_index > 0:
            ax.axvline(x=start_index-0.5, color='white', linewidth=1)

    #plt.savefig(f'{metric}.png', dpi=300, bbox_inches='tight')
    #plt.close(fig)  # close the figure
    plt.savefig(f'{metric}.png', dpi=300, bbox_inches='tight')
    plt.close(fig)  # close the figure
    # Show plot
    #plt.show()

# Now you can call this function to plot different metrics
plot_metric('r_squared_value')
plot_metric('loss')
plot_metric('test_error')
plot_metric('train_history')

In [18]:
# Log values

In [23]:
import matplotlib.pyplot as plt
import numpy as np
from mpl_toolkits.axes_grid1 import make_axes_locatable

def modelNames():

    models = ["Linear Model",
              "Wide ReLU ANN",
              "Deep ReLU ANN"
             ]
        
    for partition_num in [1,2,4,8,10]: 
        models.append(f"ABEL-Spline (z={partition_num})")
        
    for partition_num in [1,2,4,8,10]: 
        models.append(f"Spline ANN (z={partition_num})")
        
    for partition_num in [1,2,4,8,10]: 
        models.append(f"Lookup Table (z={partition_num})")

    return models

model_names = modelNames()

print(model_names)

def plot_metric(metric):
    
    # Metrics labels dictionary
    metrics_labels = {
    'loss': 'log MAE',
    'r_squared_value': '$R^2$',
    'test_error': 'log MSE',
    'train_history': 'log Training MAE'
    }
    
    instances_dict = filtered_datasets_metadata.set_index('dataset')['n_instances'].to_dict()
    features_dict = filtered_datasets_metadata.set_index('dataset')['n_features'].to_dict()

    unique_features_counts = sorted(list(set(features_dict.values())))

    sorted_dataset_names = []
    bracket_positions = {}

    for feature_count in unique_features_counts:
        datasets_with_feature_count = [dataset for dataset, features in features_dict.items() if features == feature_count]
        sorted_datasets = sorted(datasets_with_feature_count, key=lambda x: instances_dict[x])

        if not sorted_datasets:
            continue

        start_index = len(sorted_dataset_names)
        end_index = start_index + len(sorted_datasets) - 1

        bracket_positions[feature_count] = (start_index, end_index)
        
        sorted_dataset_names += [(dataset, instances_dict[dataset]) for dataset in sorted_datasets]

    data_array = np.zeros((len(model_names), len(sorted_dataset_names)))    

    mean_std_of_metric = calculate_mean_std_metric(metric)

    for i, model_name in enumerate(model_names):
        for j, (dataset_name, _) in enumerate(sorted_dataset_names):
            if model_name in mean_std_of_metric[dataset_name]:
                if metric == 'r_squared_value':
                    data_array[i, j] = mean_std_of_metric[dataset_name][model_name]['mean']
                else:
                    data_array[i, j] = np.log(mean_std_of_metric[dataset_name][model_name]['mean'])

    fig, ax = plt.subplots(figsize=(10, len(model_names)/2))
    divider = make_axes_locatable(ax)
    cax = divider.append_axes("right", size="5%", pad=0.05)

    cmap_choice = 'RdYlBu' if metric == 'r_squared_value' else 'viridis'

    # Set color limits for R^2
    clim=(-1, 1) if metric=='r_squared_value' else None

    if clim:
        im = ax.imshow(data_array, interpolation='nearest', cmap=cmap_choice, vmin=clim[0], vmax=clim[1])
    else:
        im = ax.imshow(data_array, interpolation='nearest', cmap=cmap_choice)
        
    cbar = fig.colorbar(im, cax=cax, orientation="vertical")

    cbar.set_label(metrics_labels[metric], rotation=90) 

    ax.set_yticks(np.arange(len(model_names)))
    ax.set_yticklabels(model_names)

    ax.set_xticks(np.arange(len(sorted_dataset_names)))
    ax.set_xticklabels([f"{name} ({n_instances})" for name, n_instances in sorted_dataset_names], rotation=90)

    for feature_count, (start_index, end_index) in bracket_positions.items():
        ax.annotate("", xy=(start_index-0.5,-0.5), xycoords='data',
                      xytext=(end_index+0.5,-0.5), textcoords='data',
                      annotation_clip=False,
                      arrowprops=dict(arrowstyle="-",
                                      shrinkA=0, shrinkB=0,
                                      connectionstyle="angle3"))
        ax.text((start_index+end_index)/2, -1.5, f"{feature_count}D",
                  horizontalalignment='center', verticalalignment='top')

        if start_index > 0:
            ax.axvline(x=start_index-0.5, color='white', linewidth=1)

    #plt.savefig(f'{metric}.png', dpi=300, bbox_inches='tight')
    #plt.close(fig)  # close the figure
    if metric != 'r_squared_value':
        plt.savefig(f'{metric}_log.png', dpi=300, bbox_inches='tight')
    else:
        plt.savefig(f'{metric}.png', dpi=300, bbox_inches='tight')
    plt.close(fig)  # close the figure
    # Show plot
    #plt.show()

# Now you can call this function to plot different metrics
plot_metric('r_squared_value')
plot_metric('loss')
plot_metric('test_error')
plot_metric('train_history')

['Linear Model', 'Wide ReLU ANN', 'Deep ReLU ANN', 'ABEL-Spline (z=1)', 'ABEL-Spline (z=2)', 'ABEL-Spline (z=4)', 'ABEL-Spline (z=8)', 'ABEL-Spline (z=10)', 'Spline ANN (z=1)', 'Spline ANN (z=2)', 'Spline ANN (z=4)', 'Spline ANN (z=8)', 'Spline ANN (z=10)', 'Lookup Table (z=1)', 'Lookup Table (z=2)', 'Lookup Table (z=4)', 'Lookup Table (z=8)', 'Lookup Table (z=10)']
